### Create Lecure based factors to use in Matching Experiment

Length of lectures, start time, teacher competencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sas7bdat
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf

pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
lec_group=pd.read_pickle('../Data/lektionhold.pkl')

In [ ]:
lec=pd.read_pickle('../Data/lektioner.pkl')

#Add lesson data to the lesson groups. 
lec.drop(columns=['date','lokale','gennemfrt'],inplace=True)
test=lec_group.merge(lec,on=['inst_nr','lektions_nr'],how='left')

In [ ]:
#Get teacher data for lessons
filepath='../data/macom_lektionerlaerer.sas7bdat'
teachers =pd.read_sas(filepath, format='sas7bdat',encoding='iso-8859-1')

#Clean columns
teachers.rename(columns={'InstNr':'inst_nr','LektionsNr':'lektions_nr'},inplace=True)
print(len(test))
#Merge
test=test.merge(teachers,on=['inst_nr','lektions_nr'],how='left')
print(len(test))

df=test[~test['laerer_id'].isna()]
print(len(df))

In [ ]:
#The courses from umo
filepath='../data/umo_fag.sas7bdat'
course =pd.read_sas(filepath, format='sas7bdat',encoding='iso-8859-1')

#Keep only relevant columns and rename others
course=course[['Fag','KortBetegnelse']]
course.rename(columns={'Fag':'fag_nr','KortBetegnelse':'course'},inplace=True)

#Add description to courses based on descrption and add hard_science
course_description=pd.read_excel('course_description.xlsx')
course=course.merge(course_description,on='course',how='left')

print(len(course))
course=course[~course['hard_science'].isna()]
print(len(course))
course.drop_duplicates(inplace=True)
print(len(course))

#Add descriptions to the df to allow merge with teacher competencies 
new_df=df.merge(course,on='fag_nr',how='left')
print(len(new_df['course'].isna()))

In [ ]:
# Read the Macom competencies data from SAS file
filepath = '../data/macom_kompetencer.sas7bdat'
komp = pd.read_sas(filepath, format='sas7bdat', encoding='iso-8859-1')
komp.rename(columns={'InstNr': 'inst_nr', 'KompetenceNiveau': 'competency'}, inplace=True)

# Read the descriptions for competency codes from Excel file
komp_descr = pd.read_excel('../Data/komp_koder.xlsx')
komp_descr.drop(columns=['Bekendtgørelsesafsnit', 'Kompetence betegnelse', 'AGYM', 'EGYM', 'Niveau', 'Fag'], inplace=True)
komp_descr.rename(columns={'Fagkode': 'fag_nr', 'Kompetencekode': 'KompetenceKode'}, inplace=True)

# Merge the competency codes with their descriptions
komp = komp.merge(komp_descr, on='KompetenceKode', how='left')

# Drop the unnecessary columns from the merged DataFrame
komp.drop(columns='KompetenceKode', inplace=True)

# Merge the course names with the teacher competencies
new_komp = komp.merge(course, on='fag_nr', how='left')

# Fill NA values in the competency column with 'unknown'
komp['competency'].fillna('unknown', inplace=True)

# Map the course competencies as binary values (1: competent, 0: not competent)
new_komp['competency'] = new_komp['competency'].astype(str)
competency_map = {'U': 1, 'F': 1, 'P': 0, 'I': 0}
new_komp['competency'] = new_komp['competency'].map(competency_map).fillna(0)

# Drop unnecessary columns from the new_komp DataFrame
new_komp.drop(columns=['fag_nr', 'hard_science'], inplace=True)

# Drop duplicate rows in the new_komp DataFrame
new_komp.drop_duplicates(inplace=True)

# Group the new_komp DataFrame by teacher, course, and institution number
grouped_komp = new_komp.groupby(['laerer_id', 'course', 'inst_nr'])

# Find the maximum competency value for each group
max_competency = grouped_komp['competency'].max()

# Reset the index of the max_competency DataFrame
max_competency = max_competency.reset_index()

# Merge the new_df DataFrame with the max_competency DataFrame based on teacher, course, and institution number
merged_df = new_df.merge(max_competency, on=['laerer_id', 'course', 'inst_nr'], how='left')

In [ ]:
merged_df = new_df.merge(new_komp, on=['laerer_id', 'course','inst_nr'], how='left')

temp =(merged_df.groupby(['hold_nr','start_time','end_time'],sort=False)
              .agg(**{'lektioner' :('lektions_nr','nunique')})
              .reset_index()
              )

In [ ]:
# Sort the DataFrame by 'hold_nr' and 'start_time' and reset the index
df = temp.copy()
df = df.sort_values(by=['hold_nr', 'start_time']).reset_index(drop=True)

# Check if the start time of the next row is within forty minutes of the end time of the current row.
# If so, update to the same 'block_id'. Otherwise, assign a new 'block_id' and increment the count.
def check_if_block_optimized(df):
    def update_block_id(row):
        nonlocal count
        if row['next_start_time'] < row['end_time'] + pd.Timedelta(minutes=40) and row['hold_nr'] == row['next_hold_nr']:
            block_id = count
        else:
            block_id = count
            count += 1
        return block_id

    count = 0
    # Create new columns 'next_start_time' and 'next_hold_nr' by shifting values of 'start_time' and 'hold_nr'
    df['next_start_time'] = df['start_time'].shift(-1)
    df['next_hold_nr'] = df['hold_nr'].shift(-1)

    # Apply the 'update_block_id' function to calculate the 'block_id' based on the conditions
    df['block_id'] = df.apply(update_block_id, axis=1)

    # Drop the temporary columns 'next_start_time' and 'next_hold_nr'
    df.drop(columns=['next_start_time', 'next_hold_nr'], inplace=True)

    return df

In [ ]:
df['block_id']=0
#Aplly function from above
updated_df = check_if_block_optimized(df)
updated_df.drop(columns=['lektioner'],inplace=True)

In [ ]:
#Merge back the updated to add block id
merged_df=merged_df.merge(updated_df,on=['hold_nr','start_time','end_time'],how='left')
merged_df=merged_df.sort_values(by=['hold_nr', 'start_time']).reset_index(drop=True)

In [ ]:
#Drop Gennemrfrt they are all completed anywats
merged_df.loc[~merged_df['laerer_id'].isin(new_komp['laerer_id']), 'competency'] = 'unknown'
merged_df['competency'].fillna('no', inplace=True)

# Group by hold_nr and block_id, then aggregate using the earliest start_time and latest end_time
result = merged_df.groupby(['hold_nr', 'inst_nr', 'block_id','fag_nr','niveau',
                             'hard_science','laerer_id','competency'
                            ]).agg(
    start_time=('start_time', 'min'),
    end_time=('end_time', 'max'),
    minutters_undervisning=('minutters_undervisning','sum')
).reset_index()


In [ ]:
# Extract the school year based on the 'end_date' column
result['hold_nr']=result['hold_nr'].astype(float)
result['school_year'] = result['start_time'].dt.year
result.loc[result['start_time'].dt.month < 7, 'school_year'] -= 1

# Extract the last two digits of the school year
result['school_year_suffix'] = result['school_year'] % 100

# Append the school year suffix to the hold_nr column
result['new_holdnr'] = result['hold_nr'] + result['school_year_suffix'] / 100

result=result[result['block_id'] != 0]

In [ ]:
# Group the DataFrame by 'new_holdnr' and 'block_id' and aggregate 'laerer_id' as a list
grouped_df =result.groupby(['new_holdnr', 'block_id'])['laerer_id'].agg(list).reset_index()

# Sort the values by 'block_id'
grouped_df = grouped_df.sort_values('block_id')

#Save sorted_df as what grouped_df was at this moment :) 
sorted_df=grouped_df.copy()

# Drop duplicates in 'new_holdnr' column, keeping only the first instance
grouped_df = grouped_df.drop_duplicates(subset='new_holdnr', keep='first')

In [ ]:
# Create a new column 'teacher_change' in the non-aggregated DataFrame
sorted_df['teacher_change'] = 0

# Convert grouped_df to a dictionary
teacher_sets = dict(zip(grouped_df['new_holdnr'], grouped_df['laerer_id']))

# Iterate through the rows of the non-aggregated DataFrame
for index in sorted_df.index:
    new_holdnr = sorted_df.loc[index, 'new_holdnr']
    laerer_id_list = sorted_df.loc[index, 'laerer_id']

    # Check if any teacher in the 'laerer_id' list is present in 'teacher_sets'
    teacher_in_set = any(teacher in teacher_sets[(new_holdnr)] for teacher in laerer_id_list)

    # If none of the teachers are present, set 'teacher_change' to 1 and add the entire 'laerer_id' list to 'teacher_sets'
    if not teacher_in_set:
        sorted_df.at[index, 'teacher_change'] = 1
        teacher_sets[(new_holdnr)].extend(laerer_id_list)


In [ ]:
#Count the sum of teacher changes 
teacher_changes=(sorted_df.groupby(['new_holdnr'],sort=False)
              .agg(**{'teacher_changes': ('teacher_change','sum')})
              .reset_index()
              )

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-3, 3))

# Reshape the "rounded_start" values to a 2D array for scaling
values = hours['rounded_start'].values.reshape(-1, 1)

# Scale the values using the MinMaxScaler
scaled_values = scaler.fit_transform(values)

# Assign the scaled values to a new column "normalized_start"
hours['normalized_start'] = scaled_values

In [ ]:
#Get the starttime of lectures and use it to scale, positive means later or ealier 
hours = result.drop_duplicates(subset=['new_holdnr', 'block_id'])

hours['rounded_start'] = hours['start_time'].dt.round('H').dt.hour.astype(np.int)

hours=hours.query('rounded_start > 7 & rounded_start < 17')

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-3, 3))

# Reshape the "rounded_start" values to a 2D array for scaling
values = hours['rounded_start'].values.reshape(-1, 1)

# Scale the values using the MinMaxScaler
scaled_values = scaler.fit_transform(values)

# Assign the scaled values to a new column "normalized_start"
hours['normalized_start'] = scaled_values

In [ ]:
# Calculate the absolute values of the normalized values
abs_values = np.abs(hours['normalized_start'])

# Scale the absolute values between 0 and 3
scaled_abs_values = (abs_values / abs_values.max()) * 3

# Assign the scaled absolute values to a new column "abs_normalized_start"
hours['abs_normalized_start'] = scaled_abs_values

In [ ]:
#Get sum of hours for group, avg amount hour, avg start, avg start scaled
hours = (hours.groupby(['new_holdnr'], sort=False)
              .agg(**{'sum_hours': ('minutters_undervisning', 'sum'),
                      'avg_hours': ('minutters_undervisning', 'mean'),
                      'avg_start': ('rounded_start', 'mean'),
                     'avg_start_scaled' : ('abs_normalized_start','mean')})
              .reset_index()
        )

In [ ]:
# Group the DataFrame by 'new_holdnr' and 'block_id' and aggregate 'laerer_id' as a list
grouped_lesson =result.groupby(['new_holdnr'])['laerer_id'].agg(set).reset_index()

In [ ]:
#Either competency or not
result['competency'] = np.where(result['competency']  != 1, 0, result['competency'])

In [ ]:
#First group to get if a single teacher has a competency in the course, next get the mean. This is necessary to deal with
#multi-teachers
final=(result.groupby(['new_holdnr','block_id'],sort=False)
              .agg(**{'competencies': ('competency','max')})
              .reset_index()
              )

final=(final.groupby(['new_holdnr'],sort=False)
              .agg(**{'competencies': ('competencies','mean')})
              .reset_index()
              )

In [ ]:
#Add both teacher changes, grouped lessons and start hours to the final data frame which contains competencies
merged_final=final.merge(grouped_lesson,on='new_holdnr').merge(hours,on='new_holdnr').merge(teacher_changes, on='new_holdnr')

In [ ]:
merged_final.to_pickle('../df/group_meta.pkl')